In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_uniform', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_uniform', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_uniform', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_uniform', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
he_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
from keras.optimizers import SGD
he_model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.001), metrics=["accuracy"])

In [8]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('he_1_training.log')

In [9]:
history = he_model.fit(X_train, Y_train, validation_data=[X_val, Y_val], batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 60s 1ms/step - loss: 2.0944 - acc: 0.2998 - val_loss: 1.7632 - val_acc: 0.5070
Epoch 2/20
50000/50000 [==============================] - 65s 1ms/step - loss: 1.4218 - acc: 0.6388 - val_loss: 1.0320 - val_acc: 0.7546
Epoch 3/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.8487 - acc: 0.7702 - val_loss: 0.6451 - val_acc: 0.8254
Epoch 4/20
50000/50000 [==============================] - 68s 1ms/step - loss: 0.6007 - acc: 0.8241 - val_loss: 0.4906 - val_acc: 0.8597
Epoch 5/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4838 - acc: 0.8586 - val_loss: 0.4104 - val_acc: 0.8797
Epoch 6/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4155 - acc: 0.8789 - val_loss: 0.3556 - val_acc: 0.8998
Epoch 7/20
50000/50000 [==============================] - 61s 1ms/step - loss: 0

In [10]:
(loss, accuracy) = he_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 6s 628us/step
accuracy: 94.75%


In [11]:
import pandas as pd
df = pd.read_csv('he_1_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.29980  2.094381   0.5070  1.763237
1       1  0.63882  1.421797   0.7546  1.031968
2       2  0.77016  0.848665   0.8254  0.645073
3       3  0.82406  0.600691   0.8597  0.490616
4       4  0.85862  0.483756   0.8797  0.410352
5       5  0.87886  0.415503   0.8998  0.355589
6       6  0.89256  0.370614   0.9100  0.319577
7       7  0.90224  0.338516   0.9160  0.295367
8       8  0.90860  0.313806   0.9189  0.273979
9       9  0.91458  0.293466   0.9242  0.257524
10     10  0.91908  0.276216   0.9288  0.243483
11     11  0.92324  0.261708   0.9311  0.231242
12     12  0.92734  0.248740   0.9355  0.221034
13     13  0.92994  0.237544   0.9401  0.209917
14     14  0.93340  0.227653   0.9400  0.202670
15     15  0.93560  0.218440   0.9437  0.194068
16     16  0.93784  0.210055   0.9433  0.190572
17     17  0.94040  0.202594   0.9477  0.182229
18     18  0.94188  0.195588   0.9500  0.175331
19     19  0.94476  0.189277   0.9498  0